# Start Up: 
Before Running the cell below, you must ensure that these have been run in Terminal **IN ORDER** : 
- conda update -n base -c defaults conda 

    - cd SageMaker
    
      - cd yelp-dataset-challenge-1-ds
      
         - conda env create -f environment.yml
          
            - source activate ydc1 
                
                - pip install python-decouple
                  
                  - pip install pprintpp
                  
# Spacy Installs: 

   - python -m spacy download en_core_web_lg

        - python -m spacy link en_core_web_lg en

In [15]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
import spacy
import s3
from pprintpp import pprint as pp
from sklearn.externals import joblib
nlp = spacy.load('en')

# Load in Bucket
bucket = s3.Bucket('yelpchallenge1')
# Look inside the bucket.
bucket.contents

['API/',
 'API/api.py',
 'API/api_exploration.ipynb',
 'Environments/',
 'Environments/environment.yml',
 'Flask_App/',
 'Flask_App/Pipfile',
 'Flask_App/__init__.py',
 'Flask_App/app.py',
 'Flask_App/models.py',
 'Flask_App/yelp.py',
 'Model/',
 'Model/vect_1.sav',
 'datasets/',
 'datasets/df.csv',
 'datasets/dtm.csv',
 'datasets/dtm_final.csv',
 'datasets/user.json',
 'notebooks/',
 'notebooks/data_cleanup.ipynb',
 'notebooks/official_NB.ipynb',
 'notebooks/vectorization_exploration.ipynb',
 'notebooks/yelp_data_initial_exploration.ipynb']

In [5]:
                    ### ***** DO NOT RUN. ******* #### 
                  ### ***** ALREADY INSTALLED. ****** ###

# Only have to run this once.
# Installs the .csv 'Locally' on SageMaker Instance

#bucket.get('datasets/df.csv', 'df.csv')

# Installing user.json 'Locally'
#bucket.get('datasets/user.json', 'user.json')


    # Load in Bucket
# bucket = s3.Bucket('yelpchallenge1')
    # Look inside the bucket.
# bucket.contents

# Getting Started: Imports

In [24]:
    # Read-in df.csv
df = pd.read_csv('df.csv')

    # Read-in review_df.csv
review_df = pd.read_csv('review.csv')

    # Read-in dtm_final.csv (FINAL)
dtm_final = pd.read_csv('dtm_final.csv')

    # import Vectorizer Model
vect2 = joblib.load('vect_2.sav')

# Model Prep: 

In [26]:
# Imports: 
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer

In [27]:
# Creating Training Data. 
    # X_train will include All 135,000 Rows, for 773 Vectorized Words(Columns)
    # y_train or the Target Variable  will include all 135,000 Rows, for the stars Column.     
X_train = dtm_final.iloc[:, 0:773]
y_train = dtm_final.iloc[:, 773:774]

In [29]:
# Create Pipeline Components

vect = TfidfVectorizer(stop_words='english')
clf = RandomForestClassifier()


# Define the Pipeline
pipe = Pipeline([
                 #Vectorizer
                 ('vect2', vect2), 
                 # Classifier
                 ('clf', clf)
                ])

#Tuning
parameters = {
    'vect__max_df': (0.75, 1.0),
    'vect__min_df': (.02, .05),
    'vect__max_features': (500,10000),
    'clf__n_estimators':(5, 10,),
    'clf__max_depth':(15,20)
}

grid_search = GridSearchCV(pipe, parameters, cv=5, n_jobs=5, verbose=1)
grid_search.fit(X_train, y_train)

TypeError: All intermediate steps should be transformers and implement fit and transform. '  (0, 303)	0.2928291253691394
  (0, 680)	0.11002773101546855
  (0, 592)	0.14565528128583138
  (0, 524)	0.2072584072781842
  (0, 6)	0.39740790104238377
  (0, 676)	0.2616019302844405
  (0, 77)	0.2660758124453487
  (0, 344)	0.18371335060069288
  (0, 655)	0.1444574881343549
  (0, 693)	0.08415814098566772
  (0, 207)	0.2844730508454019
  (0, 8)	0.0698290738683453
  (0, 56)	0.23871911128994652
  (0, 375)	0.08661973974387839
  (0, 209)	0.23762191417114226
  (0, 202)	0.16207525767487294
  (0, 138)	0.06902711676550477
  (0, 258)	0.26881395269403136
  (0, 123)	0.1359548153062746
  (0, 83)	0.15021774751951497
  (0, 354)	0.2145943246350761
  (0, 126)	0.2862839978064905
  (1, 303)	0.020866102788173326
  (1, 680)	0.04704142612912794
  (1, 592)	0.03113684201104495
  :	:
  (134998, 448)	0.10065064816244558
  (134998, 523)	0.21584690446362662
  (134998, 649)	0.11334342506633152
  (134998, 5)	0.11510670906958285
  